In [1]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from nn_ansatz import *
from jax.experimental.optimizers import adam
from jax import tree_util
from tqdm.notebook import trange
from jax import pmap, vmap, grad
from jax import numpy as jnp


/home/energy/amawi/miniconda3/envs/jax_cpu/lib/python3.8/site-packages/jax/experimental/optimizers.py:28: FutureWarning: jax.experimental.optimizers is deprecated, import jax.example_libraries.optimizers instead
  warnings.warn('jax.experimental.optimizers is deprecated, '
2021-11-25 11:47:33.347735: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/modules/software/binutils/2.36.1-GCCcore-10.3.0/lib:/home/modules/software/zlib/1.2.11-GCCcore-10.3.0/lib:/home/modules/software/GCCcore/10.3.0/lib64:/home/modules/software/GCCcore/10.3.0/lib


In [3]:
cfg = setup(system='HEG',
                    n_pre_it=0,
                    n_el=7,
                    n_walkers=64,
                    n_layers=2,
                    n_sh=32,
                    n_ph=8,
                    orbitals='real_plane_waves',
                    n_it=30000)

mol = SystemAnsatz(**cfg)



In [ ]:
cfg

Error: Session cannot generate requests

In [ ]:
cfg = setup(system='HEG',
                    n_pre_it=0,
                    n_el=7,
                    n_walkers=64,
                    n_layers=2,
                    n_sh=32,
                    n_ph=8,
                    orbitals='real_plane_waves',
                    n_it=30000)

log = run_vmc(cfg)

Error: Session cannot generate requests

Error: Kernel is dead

In [ ]:
keys1

Error: Kernel is dead

In [ ]:
for step in steps:
    keys, subkeys = key_gen(keys)

    walkers, acceptance, step_size = sampler(params, walkers, subkeys, step_size)
    
    grads, e_locs = grad_fn(params, walkers)

    if cfg['opt'] == 'kfac':
        grads, state = kfac_update(step, grads, state, walkers)

    state = update(step, grads, state)
    params = get_params(state)
    
    check_and_save([walkers, e_locs], ['walkers', 'e_locs'])

    steps.set_postfix(E=f'{jnp.mean(e_locs):.6f}')
    steps.refresh()

    logger.log(step,
                opt_state=state,
                params=params,
                e_locs=e_locs,
                acceptance=acceptance)

write_summary_to_cfg(cfg["csv_cfg_path"], logger.summary)
logger.walkers = walkers

In [ ]:
from utils import setup
cfg = setup(system='LiSolidBCC',
                n_pre_it=0,
                n_walkers=512,
                n_layers=2,
                n_sh=64,
                step_size=0.05,
                n_ph=32,
                scalar_inputs=False,
                orbitals='anisotropic',
                einsum=True,
                n_periodic_input=1,
                opt='adam',
                n_det=4,
                print_every=50,
                save_every=2500,
                lr=1e-4,
                n_it=30000)

compare_einsum(cfg)

In [21]:
jnp.diag(jnp.diag(jnp.eye(3, 3)))

DeviceArray([[1., 0., 0.],
             [0., 1., 0.],
             [0., 0., 1.]], dtype=float32)

In [9]:
# using routines
system = ['LiSolidBCC']
lengths = [2., 4., 6., 12., 18.]
orbitals = ['anisotropic',]
scalar_inputs = [False,]
n_ps = [1,]

e_means = []
for orbital in orbitals:
    for l in lengths:
        name = 'cell_extrapolation_%s_l%.2f' % (orbital, l)
        cfg = setup(system=system[0],
                    n_pre_it=0,
                    n_walkers=512,
                    n_layers=3,
                    n_sh=32,
                    step_size=0.02,
                    n_ph=8,
                    scalar_inputs=False,
                    orbitals=orbital,
                    n_periodic_input=1,
                    opt='adam',
                    n_det=4,
                    print_every=100,
                    save_every=2500,
                    lr=1e-4,
                    n_it=5000,
                    name=name)
        cfg['unit_cell_length'] = l
        log = run_vmc(cfg)
        e_mean = log.summary['e_mean_mean']
        e_means.append(e_mean)



s32_p8_l3_det4 adam_1lr-4_1d-3_1nc-4_m512_ cell_extrapolation_anisotropic_l2.00
version 		 081021
seed 		 369
n_devices 		 1
save_every 		 2500
print_every 		 100
exp_dir 		 /home/amawi/projects/nn_ansatz/src/experiments/LiSolid/081021/cell_extrapolation_anisotropic_l2.00/adam_1lr-4_1d-3_1nc-4_m512_s32_p8_l3_det4/run0
events_dir 		 /home/amawi/projects/nn_ansatz/src/experiments/LiSolid/081021/cell_extrapolation_anisotropic_l2.00/adam_1lr-4_1d-3_1nc-4_m512_s32_p8_l3_det4/run0/events
models_dir 		 /home/amawi/projects/nn_ansatz/src/experiments/LiSolid/081021/cell_extrapolation_anisotropic_l2.00/adam_1lr-4_1d-3_1nc-4_m512_s32_p8_l3_det4/run0/models
opt_state_dir 		 /home/amawi/projects/nn_ansatz/src/experiments/LiSolid/081021/cell_extrapolation_anisotropic_l2.00/adam_1lr-4_1d-3_1nc-4_m512_s32_p8_l3_det4/run0/models/opt_state
pre_path 		 /home/amawi/projects/nn_ansatz/src/experiments/LiSolid/pretrained/s32_p8_l3_det4_1lr-4_i0.pk
timing_dir 		 /home/amawi/projects/nn_ansatz/src/experiments/

training:   0%|          | 1/5001 [00:00<?, ?it/s]

step 100 | e_mean 5.8891 | e_std 16.8201 | e_mean_mean 6.2794 | acceptance 0.5254 | t_per_it 0.0356 |
step 200 | e_mean 5.7627 | e_std 13.5526 | e_mean_mean 5.9507 | acceptance 0.4916 | t_per_it 0.0381 |
step 300 | e_mean 5.3059 | e_std 13.9966 | e_mean_mean 5.6183 | acceptance 0.4959 | t_per_it 0.0382 |
step 400 | e_mean 6.0144 | e_std 9.6275 | e_mean_mean 5.7116 | acceptance 0.4871 | t_per_it 0.0382 |
step 500 | e_mean 4.1402 | e_std 6.7855 | e_mean_mean 4.5146 | acceptance 0.5078 | t_per_it 0.0381 |
step 600 | e_mean 4.1454 | e_std 18.2803 | e_mean_mean 4.1704 | acceptance 0.4908 | t_per_it 0.0381 |
step 700 | e_mean 5.2978 | e_std 10.0753 | e_mean_mean 4.6564 | acceptance 0.5047 | t_per_it 0.0385 |
step 800 | e_mean 4.5718 | e_std 5.8287 | e_mean_mean 4.2432 | acceptance 0.4992 | t_per_it 0.0385 |
step 900 | e_mean 4.4916 | e_std 8.0385 | e_mean_mean 4.0622 | acceptance 0.5047 | t_per_it 0.0384 |
step 1000 | e_mean 4.0054 | e_std 8.3852 | e_mean_mean 4.0255 | acceptance 0.5117 | t_

training:   0%|          | 1/5001 [00:00<?, ?it/s]

step 100 | e_mean -3.7176 | e_std 4.6901 | e_mean_mean -3.6686 | acceptance 0.6879 | t_per_it 0.0346 |
step 200 | e_mean -4.1188 | e_std 4.9976 | e_mean_mean -4.1487 | acceptance 0.4951 | t_per_it 0.0359 |
step 300 | e_mean -4.6208 | e_std 4.6106 | e_mean_mean -4.5200 | acceptance 0.4971 | t_per_it 0.0366 |
step 400 | e_mean -5.2458 | e_std 4.6282 | e_mean_mean -4.9305 | acceptance 0.4900 | t_per_it 0.0366 |
step 500 | e_mean -5.0656 | e_std 3.2627 | e_mean_mean -5.1085 | acceptance 0.5074 | t_per_it 0.0362 |
step 600 | e_mean -5.2301 | e_std 2.9122 | e_mean_mean -5.1274 | acceptance 0.4879 | t_per_it 0.0361 |
step 700 | e_mean -5.1892 | e_std 2.8058 | e_mean_mean -5.2871 | acceptance 0.5051 | t_per_it 0.0361 |
step 800 | e_mean -5.1685 | e_std 3.4644 | e_mean_mean -5.3230 | acceptance 0.4889 | t_per_it 0.0363 |
step 900 | e_mean -5.4281 | e_std 2.2979 | e_mean_mean -5.3715 | acceptance 0.4984 | t_per_it 0.0363 |
step 1000 | e_mean -5.2998 | e_std 2.5123 | e_mean_mean -5.3996 | accepta

training:   0%|          | 1/5001 [00:00<?, ?it/s]

step 100 | e_mean -5.2849 | e_std 3.4477 | e_mean_mean -5.2070 | acceptance 0.7424 | t_per_it 0.0381 |
step 200 | e_mean -5.9378 | e_std 3.7910 | e_mean_mean -5.8801 | acceptance 0.5141 | t_per_it 0.0384 |
step 300 | e_mean -6.1578 | e_std 3.3906 | e_mean_mean -6.3210 | acceptance 0.5096 | t_per_it 0.0391 |
step 400 | e_mean -6.5130 | e_std 2.6586 | e_mean_mean -6.4526 | acceptance 0.4914 | t_per_it 0.0391 |
step 500 | e_mean -6.5399 | e_std 2.5840 | e_mean_mean -6.5537 | acceptance 0.5182 | t_per_it 0.0390 |
step 600 | e_mean -6.6020 | e_std 2.4132 | e_mean_mean -6.6595 | acceptance 0.4924 | t_per_it 0.0391 |
step 700 | e_mean -6.5995 | e_std 2.0006 | e_mean_mean -6.6624 | acceptance 0.4904 | t_per_it 0.0390 |
step 800 | e_mean -6.8579 | e_std 3.3405 | e_mean_mean -6.6997 | acceptance 0.4885 | t_per_it 0.0389 |
step 900 | e_mean -6.7412 | e_std 1.9233 | e_mean_mean -6.7279 | acceptance 0.4908 | t_per_it 0.0390 |
step 1000 | e_mean -7.1283 | e_std 4.5345 | e_mean_mean -6.7365 | accepta

training:   0%|          | 1/5001 [00:00<?, ?it/s]

step 100 | e_mean -5.0802 | e_std 2.9715 | e_mean_mean -5.0960 | acceptance 0.7932 | t_per_it 0.0377 |
step 200 | e_mean -5.6856 | e_std 5.1371 | e_mean_mean -5.4111 | acceptance 0.6250 | t_per_it 0.0387 |
step 300 | e_mean -5.6692 | e_std 3.0718 | e_mean_mean -5.7061 | acceptance 0.4996 | t_per_it 0.0389 |
step 400 | e_mean -6.1102 | e_std 2.6608 | e_mean_mean -6.1062 | acceptance 0.4889 | t_per_it 0.0387 |
step 500 | e_mean -6.3427 | e_std 2.8793 | e_mean_mean -6.4825 | acceptance 0.5156 | t_per_it 0.0391 |
step 600 | e_mean -6.6737 | e_std 2.8598 | e_mean_mean -6.6563 | acceptance 0.4920 | t_per_it 0.0391 |
step 700 | e_mean -7.0534 | e_std 3.1995 | e_mean_mean -6.8385 | acceptance 0.5045 | t_per_it 0.0392 |
step 800 | e_mean -6.9495 | e_std 2.8286 | e_mean_mean -6.8923 | acceptance 0.4850 | t_per_it 0.0391 |
step 900 | e_mean -7.1450 | e_std 2.2172 | e_mean_mean -6.9905 | acceptance 0.4840 | t_per_it 0.0391 |
step 1000 | e_mean -6.9568 | e_std 2.0335 | e_mean_mean -6.9943 | accepta

training:   0%|          | 1/5001 [00:00<?, ?it/s]

step 100 | e_mean -4.7240 | e_std 3.2381 | e_mean_mean -4.6541 | acceptance 0.7965 | t_per_it 0.0385 |
step 200 | e_mean -5.3854 | e_std 3.4719 | e_mean_mean -5.2747 | acceptance 0.6383 | t_per_it 0.0401 |
step 300 | e_mean -5.3707 | e_std 3.0301 | e_mean_mean -5.3619 | acceptance 0.5080 | t_per_it 0.0395 |
step 400 | e_mean -5.8044 | e_std 3.3492 | e_mean_mean -5.5814 | acceptance 0.4885 | t_per_it 0.0399 |
step 500 | e_mean -5.9643 | e_std 2.6648 | e_mean_mean -6.0918 | acceptance 0.5043 | t_per_it 0.0397 |
step 600 | e_mean -6.5549 | e_std 3.6277 | e_mean_mean -6.3887 | acceptance 0.4926 | t_per_it 0.0396 |
step 700 | e_mean -6.6757 | e_std 3.1546 | e_mean_mean -6.6663 | acceptance 0.4982 | t_per_it 0.0395 |
step 800 | e_mean -7.0942 | e_std 3.4489 | e_mean_mean -6.7696 | acceptance 0.4785 | t_per_it 0.0395 |
step 900 | e_mean -6.9700 | e_std 3.0241 | e_mean_mean -6.9070 | acceptance 0.4969 | t_per_it 0.0395 |
step 1000 | e_mean -7.3287 | e_std 3.6990 | e_mean_mean -6.9318 | accepta

[array(0.45078853), array(-2.05238247), array(-2.6465919), array(-2.81724381), array(-2.900424)]


In [8]:
import matplotlib.pyplot as plt
# 2 3 4 5 10
gpaw_means = [-6.639510, -0.076957, -0.00276, 0.006213]
gpaw_lengths = [2., 4., 5., 10.]
ev = 27.2113245
gpaw_means = [(x -77.136551) / ev for x in gpaw_means]
plt.plot(lengths, e_means)
plt.plot(gpaw_lengths, gpaw_means)
print(e_means)
plt.show()

In [4]:
log = run_vmc(config)
# save_pk(log.params, 'params.pk')

System: 
 n_atoms = 2 
 n_up    = 3 
 n_down  = 3 
 n_el    = 6 

Ansatz: 
 n_layers = 3 
 n_det    = 4 
 n_sh     = 64 
 n_ph     = 16 

Cell: 
 real_basis: 
 [[6.63 0.   0.  ]
 [0.   6.63 0.  ]
 [0.   0.   6.63]] 
 reciprocal_basis: 
 [[0.94769007 0.         0.        ]
 [0.         0.94769007 0.        ]
 [0.         0.         0.94769007]] 
 real_cut         = 6.00 
 reciprocal_cut   = 6 
 kappa            = 0.50 
 volume           = 291.43 
 min_cell_width   = 6.63 
 n_periodic_input = 1 
 unit_cell_length = 6.63 

converged SCF energy = -12.1628037018806
creating wf
sampling no infs, this could take a while
end sampling no infs


training:   0%|          | 1/7501 [00:00<?, ?it/s]

step 100 | e_mean -9.1831 | e_std 3.3782 | e_mean_mean -9.1150 | acceptance 0.5057 | t_per_it 0.0684 |
step 200 | e_mean -9.6069 | e_std 3.0851 | e_mean_mean -9.4766 | acceptance 0.4895 | t_per_it 0.0685 |
step 300 | e_mean -9.8351 | e_std 2.9581 | e_mean_mean -9.9441 | acceptance 0.5074 | t_per_it 0.0689 |
step 400 | e_mean -10.2764 | e_std 3.1788 | e_mean_mean -10.1491 | acceptance 0.5111 | t_per_it 0.0688 |
step 500 | e_mean -10.8160 | e_std 3.1381 | e_mean_mean -10.5383 | acceptance 0.5025 | t_per_it 0.0687 |


KeyboardInterrupt: 

In [23]:
walkers = None
logger = Logging(**cfg)

keys = rnd.PRNGKey(cfg['seed'])
if bool(os.environ.get('DISTRIBUTE')) is True:
    keys = rnd.split(keys, cfg['n_devices']).reshape(cfg['n_devices'], 2)

mol = SystemAnsatz(**cfg)

vwf = create_wf(mol)
params = initialise_params(mol, keys)
pwf = pmap(create_wf(mol), in_axes=(None, 0))
pwf_grad = grad(lambda x, y: pwf(x, y).sum(), argnums=(0,1))

sampler = create_sampler(mol, vwf)

if cfg['load_pretrain']:
    params, walkers = load_pk(cfg['pre_path'])
elif cfg['pretrain']:
    params, walkers = pretrain_wf(mol, **cfg)

if walkers is None:
    walkers = initialise_walkers(mol, vwf, sampler, params, keys, walkers=walkers)

grad_fn = create_grad_function(mol, vwf)

if cfg['opt'] == 'kfac':
    update, get_params, kfac_update, state = kfac(mol, params, walkers, cfg['lr'], cfg['damping'], cfg['norm_constraint'])
elif cfg['opt'] == 'adam':
    init, update, get_params = adam(cfg['lr'])
    update = jit(update)
    state = init(params)
else:
    exit('Optimiser not available')

steps = trange(1, cfg['n_it']+1, initial=1, total=cfg['n_it']+1, desc='training', disable=None)
step_size = split_variables_for_pmap(cfg['n_devices'], cfg['step_size'])

ke = pmap(create_local_kinetic_energy(vwf), in_axes=(None, 0))
pe = pmap(create_potential_energy(mol), in_axes=(0, None, None))
pmap_compute_ae_vectors_periodic_i = lambda x, y: compute_ae_vectors_periodic_i(x, y, mol.unit_cell_length)
pmap_compute_ae_vectors_periodic_i = pmap(vmap(pmap_compute_ae_vectors_periodic_i, in_axes=(0, None)), in_axes=(0, None))
pmap_compute_ee_vectors_i = pmap(vmap(compute_ee_vectors_i, in_axes=(0,)), in_axes=(0,))



System: 
 n_atoms = 2 
 n_up    = 3 
 n_down  = 3 
 n_el    = 6 

Ansatz: 
 n_layers = 2 
 n_det    = 2 
 n_sh     = 32 
 n_ph     = 8 

Cell: 
 real_basis: 
 [[6.63 0.   0.  ]
 [0.   6.63 0.  ]
 [0.   0.   6.63]] 
 reciprocal_basis: 
 [[0.94769007 0.         0.        ]
 [0.         0.94769007 0.        ]
 [0.         0.         0.94769007]] 
 real_cut         = 6.00 
 reciprocal_cut   = 6 
 kappa            = 0.50 
 volume           = 291.43 
 min_cell_width   = 6.63 
 n_periodic_input = 3 
 unit_cell_length = 6.63 

converged SCF energy = -12.1628037018806
creating wf
creating wf
sampling no infs, this could take a while
end sampling no infs


training:   0%|          | 1/5001 [00:00<?, ?it/s]

In [24]:
for step in steps:
    keys, subkeys = key_gen(keys)

    walkers, acceptance, step_size = sampler(params, walkers, subkeys, step_size)

    gparam, gwalker = pwf_grad(params, walkers)
    # gparam_nan, gwalker_nan = check_if_nan(gparam, 'x'), check_if_nan(gwalker, 'x')

    pote = pe(walkers, mol.r_atoms, mol.z_atoms)
    pote_nan = check_if_nan(pote, 'x')

    kine = ke(params, walkers)
    kine_nan = check_if_nan(kine,'x')

    ae_vectors = pmap_compute_ae_vectors_periodic_i(walkers, mol.r_atoms)
    ee_vectors = pmap_compute_ee_vectors_i(walkers)
    min_im_ee_vectors = apply_minimum_image_convention(ee_vectors, mol.unit_cell_length)
    min_im_ae_vectors = apply_minimum_image_convention(ae_vectors, mol.unit_cell_length)

    if kine_nan:
        print('nan in kinetic')
        break

    grads, e_locs = grad_fn(params, walkers)

    if cfg['opt'] == 'kfac':
        grads, state = kfac_update(step, grads, state, walkers)

    state = update(step, grads, state)
    params = get_params(state)
    # stop = capture_nan(params, 'params', stop)

    steps.set_postfix(E=f'{jnp.mean(e_locs):.6f}')
    steps.refresh()

    logger.log(step,
                opt_state=state,
                params=params,
                e_locs=e_locs,
                acceptance=acceptance[0])

    # if stop:
        # exit()

logger.walkers = walkers

TypeError: isnan requires ndarray or scalar arguments, got <class 'collections.OrderedDict'> at position 0.

In [ ]:
# Pretrain

params, walkers = pretrain_wf(params,
                              wf,
                              wf_orbitals,
                              mol,
                              walkers,
                              n_it=n_pre_it,
                              lr=1e-4,
                              n_eq_it=n_pre_it)

In [10]:
# vmc loop with kfac

grad_fn = create_grad_function(wf, mol)


update, get_params, kfac_update, state = kfac(kfac_wf, wf, mol, params, walkers, d0s,
                                                  lr=lr,
                                                  damping=damping,
                                                  norm_constraint=norm_constraint)


steps = trange(0, n_it, initial=0, total=n_it, desc='training', disable=None)
for step in steps:
    key, subkey = rnd.split(key)

    walkers, acceptance, step_size = sampler(params, walkers, d0s, subkey, step_size)

    grads, e_locs = grad_fn(params, walkers, d0s)

    grads, state = kfac_update(step, grads, state, walkers, d0s)

    state = update(step, grads, state)
    params = get_params(state)

    steps.set_postfix(E=f'{jnp.mean(e_locs):.6f}')
    steps.refresh()
    

print('exit')

exit


In [9]:
# periodic boundaries test

for (name, method) in zip(('LiSolid'), ('kfac')):
    config = setup(system=name,
                   pretrain=True,
                   n_pre_it=501,
                   n_walkers=2048,
                   n_layers=2,
                   n_sh=64,
                   n_ph=16,
                   opt=method,
                   n_det=8,
                   print_every=100,
                   save_every=5000,
                   lr=1e-4,
                   n_it=10000,
                   norm_constraint=1e-4,
                   damping=1e-3,
                   exp=True)

    run_vmc(**config)


Pretrain path does not exist, can't load


NameError: name 'exit' is not defined

In [3]:
config = setup(system='LiSolid',
               pretrain=True,
               n_pre_it=501,
               n_walkers=2048,
               n_layers=2,
               n_sh=64,
               n_ph=16,
               opt='kfac',
               n_det=8,
               print_every=100,
               save_every=5000,
               lr=1e-3,
               n_it=1000,
               norm_constraint=1e-4,
               damping=1e-3,
               exp=True,
               name='sampler_fix')

run_vmc(**config)

version 		 200421
seed 		 369
save_every 		 5000
print_every 		 100
exp_dir 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolid/sampler_fix/kfac_1lr-3_1d-3_1nc-4_m2048_s64_p16_l2_det8/run4
events_dir 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolid/sampler_fix/kfac_1lr-3_1d-3_1nc-4_m2048_s64_p16_l2_det8/run4/events
models_dir 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolid/sampler_fix/kfac_1lr-3_1d-3_1nc-4_m2048_s64_p16_l2_det8/run4/models
opt_state_dir 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolid/sampler_fix/kfac_1lr-3_1d-3_1nc-4_m2048_s64_p16_l2_det8/run4/models/opt_state
pre_path 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolid/pretrained/s64_p16_l2_det8_1lr-4_i501.pk
timing_dir 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolid/sampler_fix/kfac_1lr-3_1d-3_1nc-4_m2048_s64_p16_l2_det8/run4/events/timing
system 		 LiSolid
r_atoms 		 [[0.5 0.5 0.5]]
z_atoms 		 [3.]
n_el 		 3
n_el_atoms 		 [3]
periodic_boundaries 		 True
cell_basis 		 [[-0

training:   0%|          | 0/1000 [00:00<?, ?it/s]

TypeError: div got incompatible shapes for broadcasting: (2048, 3, 3), (2048, 3, 14).